In [ ]:
# 필수 패키지 설치 (langchain-community 등)
!pip install -q langchain-community langchain-core sentence-transformers faiss-cpu transformers torch

# 📊 00.05: RAG 평가 맛보기
Naive RAG가 어떤 기준에서 부족한지 직접 체험해 봅니다.

In [ ]:
# 1. 평가 환경 준비
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

RAW_DOCS = [
    {"title": "파이썬 역사", "content": "파이썬은 1991년 귀도 반 로섬이 만든 언어입니다.", "category": "programming"},
    {"title": "머신러닝 개요", "content": "머신러닝은 데이터를 학습해 패턴을 찾는 기술입니다.", "category": "ai"},
    {"title": "RAG 소개", "content": "RAG 시스템은 검색과 생성을 결합해 관련 문서를 기반으로 답변합니다.", "category": "ai"},
    {"title": "서울 여행", "content": "서울은 문화유산과 최신 트렌드가 공존하는 도시입니다.", "category": "travel"},
]

DOCUMENTS = [Document(page_content=d["content"], metadata=d) for d in RAW_DOCS]
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(DOCUMENTS, embedding_model)
print("✅ 평가용 벡터스토어 생성 완료")

In [ ]:
# 2. 평가용 질문 세트 정의
EVAL_QUESTIONS = [
    {"question": "파이썬을 만든 사람은 누구야?", "expected_kw": "귀도", "type": "factual"},
    {"question": "RAG는 무엇을 결합해?", "expected_kw": "검색", "type": "definition"},
    {"question": "머신러닝은 무슨 기술이야?", "expected_kw": "패턴", "type": "definition"},
    {"question": "서울 여행은 어떨까?", "expected_kw": "문화", "type": "experience"},
]
print("평가 질문 개수:", len(EVAL_QUESTIONS))

In [ ]:
# 3. Top-k 구성 결과 및 지표 계산
from difflib import SequenceMatcher
import numpy as np
import pandas as pd

results_summary = []

for item in EVAL_QUESTIONS:
    question = item['question']
    expected = item['expected_kw']
    retrieved = vectorstore.similarity_search(question, k=3)
    coverage = [expected in doc.page_content for doc in retrieved]
    similarity = [SequenceMatcher(None, question.lower(), doc.page_content.lower()).ratio() for doc in retrieved]
    results_summary.append({
        'question': question,
        'type': item['type'],
        'top1_hit': coverage[0],
        'recall@3': float(np.mean(coverage)),
        'avg_sim': round(float(np.mean(similarity)), 2)
    })

summary_df = pd.DataFrame(results_summary)
print('🔍 상위 문서 성능 요약')
display(summary_df)

print('해석 예시:')
for row in summary_df.to_dict('records'):
    msg = " - '{q}' ({t}) -> Top1 Hit: {hit}, Recall@3: {rc:.0f}%, 평균 문자유사도: {sim:.2f}".format(
        q=row['question'], t=row['type'], hit=row['top1_hit'], rc=row['recall@3']*100, sim=row['avg_sim'])
    print(msg)
print('해석 방법:')
print('• Top1 Hit=True 이더라도 Recall@3가 33%라면, 상위 3개 중 관련 문서는 1개뿐임을 의미합니다.')
print('• 평균 문자유사도가 0.1~0.3이면 질문-문서가 표면적으로만 조금 비슷하다는 뜻 → 의미 연구 필요.')
print('• 따라서 나이브 RAG는 첫 문서만 운 좋게 맞을 뿐, 후보군이 취약하다는 사실을 보여줍니다.')


In [ ]:
# 4. 평가 지표 요약
accuracy = summary_df['top1_hit'].mean() * 100
recall3 = summary_df['recall@3'].mean() * 100
avg_similarity = summary_df['avg_sim'].mean()

print('📈 요약 지표')
print('- Top1 정확도: {:.1f}% (1위 문서가 정답을 포함했는지)'.format(accuracy))
print('- Recall@3 평균: {:.1f}% (상위 3개 중 관련 문서 비율)'.format(recall3))
print('- 평균 문자유사도: {:.2f} (0~1, 문자열 수준 유사도)'.format(avg_similarity))

if accuracy < 80:
    print('⚠️ 키워드가 빠진 결과가 있으므로 검색 강화 필요 (다음 고급 실습에서 해결)')

In [ ]:
# 5. LLM 기반 RAG 평가 시스템 (RAGAs 스타일)
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

exaone_tokenizer = AutoTokenizer.from_pretrained('LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct', trust_remote_code=True)
exaone_model = AutoModelForCausalLM.from_pretrained('LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct', trust_remote_code=True, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32, device_map='auto')

# 디버깅 모드 (True로 설정하면 모델 응답 출력)
DEBUG_MODE = True

def evaluate_with_llm(question, context, answer, metric_type):
    """LLM을 사용하여 RAG 답변을 평가 (개선된 버전)"""
    
    if metric_type == "faithfulness":
        prompt = f"""답변이 컨텍스트에만 근거하는지 평가하세요.

컨텍스트: {context}
답변: {answer}

평가 예시:
- 컨텍스트에 있는 내용만 사용 → 5점
- 컨텍스트 내용 + 약간의 외부 지식 → 3점
- 환각이 많음 → 0점

답변이 컨텍스트만 사용했나요?
반드시 "점수: X" 형식으로 답하세요 (X는 0, 1, 2, 3, 4, 5 중 하나):"""

    elif metric_type == "relevancy":
        prompt = f"""답변이 질문에 직접 답하는지 평가하세요.

질문: {question}
답변: {answer}

평가 예시:
- 질문에 정확히 답함 → 5점
- 질문과 약간 관련 → 3점
- 질문과 무관 → 0점

답변이 질문에 답하나요?
반드시 "점수: X" 형식으로 답하세요 (X는 0, 1, 2, 3, 4, 5 중 하나):"""

    elif metric_type == "correctness":
        prompt = f"""답변이 정확한지 평가하세요.

질문: {question}
정답 컨텍스트: {context}
답변: {answer}

평가 예시:
- 완벽히 정확 → 5점
- 대체로 정확 → 3점
- 틀림 → 0점

답변이 정확한가요?
반드시 "점수: X" 형식으로 답하세요 (X는 0, 1, 2, 3, 4, 5 중 하나):"""
    
    else:
        return 0.0
    
    inputs = exaone_tokenizer(prompt, return_tensors='pt').to(exaone_model.device)
    input_length = inputs.input_ids.shape[1]
    
    with torch.no_grad():
        outputs = exaone_model.generate(
            **inputs, 
            max_new_tokens=30,
            temperature=0.1,  # 더 일관된 출력
            do_sample=True,
            pad_token_id=exaone_tokenizer.eos_token_id
        )
    
    generated_tokens = outputs[0][input_length:]
    response = exaone_tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    # 디버깅: 모델 응답 출력
    if DEBUG_MODE:
        print(f"  [{metric_type}] 모델 응답: {response[:50]}")
    
    # 개선된 숫자 추출: "점수: 5", "5점", "5" 등 다양한 형식 지원
    score_patterns = [
        r'점수[:\s]*([0-5])',  # "점수: 5" or "점수 5"
        r'([0-5])\s*점',       # "5점"
        r'^([0-5])$',          # "5"
        r'([0-5])',            # 아무 숫자나
    ]
    
    score = None
    for pattern in score_patterns:
        match = re.search(pattern, response)
        if match:
            score = int(match.group(1)) / 5.0  # 0~1 스케일로 정규화
            break
    
    if score is None:
        if DEBUG_MODE:
            print(f"  ⚠️ 파싱 실패! 기본값 0.5 사용")
        score = 0.5
    
    return score

# 전체 평가 실행
print('🔍 LLM 기반 RAG 평가 시작 (RAGAs 스타일)')
print('-' * 80)

eval_results = []

for item in EVAL_QUESTIONS:
    question = item['question']
    
    # 컨텍스트 검색
    retrieved_docs = vectorstore.similarity_search(question, k=1)
    context = retrieved_docs[0].page_content
    
    # 답변 생성
    answer_prompt = f"질문: {question}\n컨텍스트: {context}\n답변:"
    inputs = exaone_tokenizer(answer_prompt, return_tensors='pt').to(exaone_model.device)
    input_length = inputs.input_ids.shape[1]
    
    with torch.no_grad():
        output = exaone_model.generate(**inputs, max_new_tokens=80)
    
    generated_tokens = output[0][input_length:]
    answer = exaone_tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    
    if DEBUG_MODE:
        print(f"\n질문: {question}")
        print(f"답변: {answer[:60]}")
    
    # 3가지 메트릭 평가
    faithfulness = evaluate_with_llm(question, context, answer, "faithfulness")
    relevancy = evaluate_with_llm(question, context, answer, "relevancy")
    correctness = evaluate_with_llm(question, context, answer, "correctness")
    
    avg_score = (faithfulness + relevancy + correctness) / 3
    
    eval_results.append({
        'question': question[:30] + '...' if len(question) > 30 else question,
        'context': context[:50] + '...' if len(context) > 50 else context,
        'answer': answer[:40] + '...' if len(answer) > 40 else answer,
        'faithfulness': round(faithfulness, 2),
        'relevancy': round(relevancy, 2),
        'correctness': round(correctness, 2),
        'avg_score': round(avg_score, 2)
    })
    
    print(f"✓ 평균: {avg_score:.2f}")

# 결과 DataFrame 출력
eval_df = pd.DataFrame(eval_results)
print('\n📊 LLM 기반 RAG 평가 결과 (0~1 스케일)')
display(eval_df)

print('\n해석 가이드:')
print('- faithfulness: 답변이 컨텍스트만 사용했는지 (환각 체크)')
print('- relevancy: 답변이 질문과 직접 관련있는지')
print('- correctness: 답변이 정확하고 완전한지')
print('- avg_score: 3가지 메트릭 평균')
print('- 0.7 이상: 우수, 0.5~0.7: 보통, 0.5 미만: 개선 필요')
print(f'\n💡 디버그 모드: {DEBUG_MODE} (셀 상단에서 False로 변경 가능)')

In [ ]:
# 6. Q&A 요약
print('🎉 00.05 RAG 평가 맛보기 완료!')
print('- 상위 문서 순위와 키워드 커버리지를 통해 나이브 RAG 한계를 확인했습니다.')
print('- Recall@3, 문자 유사도 등 간단 지표로 성능을 수치화했습니다.')
print('- LLM 기반 자체 평가로 Faithfulness, Relevancy, Correctness를 측정했습니다.')
print('➡️ 다음 실습에서 쿼리 개선, 메타데이터 필터링, 하이브리드 검색을 적용해 보세요.')